In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
path = "../weee/dataset/"

In [61]:
# df_all = pd.DataFrame({'user_id': [], 'x': [], 'y': [], 'z': []})
# df_MET = pd.DataFrame({'user_id': [], 'met_value_mean_values': []})

# df_labels = pd.read_csv(f'{path}/VO2.csv')

# for i in range(1,2):

#     # Two subjects should neglected because of the study issues
#     if i+1 in [6, 14]:
#         continue

#     # Read CSV file
#     df = pd.read_csv(f'{path}/P{i+1:02d}/E4/ACC.csv', header=None, names=['x', 'y', 'z'])

#     # Take the first row as the start timestamp
#     start = df.iloc[0, :]['x']*1000

#     # Convert above time stamp to datetime
#     start = pd.to_datetime(start, unit='ms')

#     # Remove first two rows from the df
#     df = df.iloc[2:, :]

#     # Create a timestamp column using start timestamp and with the sampling rate of 32 Hz
#     df['timestamp'] = pd.date_range(start=start, periods=df.shape[0], freq='31.25ms')

#     # Make the timestamp column as the index
#     df.set_index('timestamp', inplace=True)

#     # resample the data to 50Hz with interpolation
#     df = df.resample('20ms').ffill()

#     # get the dataframe only for user_id = i+1 from df_labels
#     df_label = df_labels[df_labels['user_id'] == i+1]

#     # Get the first and last start_timestamp values of particular user_id from the df_labels
#     start_timestamp = pd.to_datetime(df_labels[df_labels['user_id'] == i+1]['start_timestamp'].iloc[0])
#     end_timestamp = pd.to_datetime(df_labels[df_labels['user_id'] == i+1]['end_timestamp'].iloc[-1])

#     # Get the data only between start_timestamp and end_timestamp
#     df = df[start_timestamp:end_timestamp]

#     # Get list of start_timestamp of the missing values of the column "met_value_mean_values" of df_label
#     missing_start_timestamp = df_label.loc[df_label['met_value_mean_values'].isna(), ['start_timestamp', 'end_timestamp']]

#     # Remove the missing values from the df using the above list
#     for j in range(missing_start_timestamp.shape[0]):
#         df = df.drop(pd.date_range(start=missing_start_timestamp.iloc[j, 0], end=missing_start_timestamp.iloc[j, 1], freq='20ms'))

#     # Divide all columns by 64
#     df = df / 64

#     # Add a column for user_id
#     df['user_id'] = i+1

#     # Make index as a range column and make timestamp as the first column
#     df.reset_index(inplace=True)

#     # Neglect the last df.shape[0] % 64 rows
#     if df.shape[0] % 64 != 0:
#         df = df.iloc[:-(df.shape[0] % 64), :]

#     # Append to the dataframe
#     df_all = pd.concat([df_all, df.loc[:, ['user_id', 'x', 'y', 'z']]])

#     # Make start_timestamp as datetime format
#     df_label.loc[:, 'start_timestamp'] = pd.to_datetime(df_label.loc[:,'start_timestamp'])

#     # Make the start_timestamp as index
#     df_label.set_index('start_timestamp', inplace=True)

#     # Remove the missing values from the df_label using the above list
#     for j in range(missing_start_timestamp.shape[0]):
#         df_label = df_label.drop(pd.date_range(start=missing_start_timestamp.iloc[j, 0], end=missing_start_timestamp.iloc[j, 0], freq='2s'))

#     # Current data in 0.5 Hz, so resample it to 1 Hz
#     df_label = df_label.resample('1s').ffill()

#     # Print the shape of df_label
#     print(df_label.shape)

#     # Append to the dataframe
#     df_MET = pd.concat([df_MET, df_label.loc[:, ['user_id', 'met_value_mean_values']]])

#     # Print the finished user_id
#     print(f'Finished {i+1}')

# # Save to CSV file
# df_all.to_csv(f'{path}/combined_e4_acc.csv', index=False)

# # Save to CSV file
# df_MET.to_csv(f'{path}/combined_met_labels.csv', index=False)

# # Print the shape of df_all
# print(df_all.shape)

In [26]:
df_all = pd.DataFrame({'timestamp': [], 'user_id': [], 'x': [], 'y': [], 'z': [], 'met_value_mean_values': []})

df_labels = pd.read_csv(f'{path}/VO2.csv')

for i in range(17):

    # Two subjects should neglected because of the study issues
    if i+1 in [6, 14]:
        continue

    # Read CSV file
    df = pd.read_csv(f'{path}/P{i+1:02d}/E4/ACC.csv', header=None, names=['x', 'y', 'z'])

    # Take the first row as the start timestamp
    start = df.iloc[0, :]['x']*1000

    # Convert above time stamp to datetime
    start = pd.to_datetime(start, unit='ms')

    # Remove first two rows from the df
    df = df.iloc[2:, :]

    # Create a timestamp column using start timestamp and with the sampling rate of 32 Hz
    df['timestamp'] = pd.date_range(start=start, periods=df.shape[0], freq='31.25ms')

    # Make the timestamp column as the index
    df.set_index('timestamp', inplace=True)

    # resample the data to 50Hz with interpolation
    df = df.resample('20ms').ffill()

    # get the dataframe only for user_id = i+1 from df_labels
    df_label = df_labels[df_labels['user_id'] == i+1]

    # Get the first and last start_timestamp values of particular user_id from the df_labels
    start_timestamp = pd.to_datetime(df_labels[df_labels['user_id'] == i+1]['start_timestamp'].iloc[0])
    end_timestamp = pd.to_datetime(df_labels[df_labels['user_id'] == i+1]['end_timestamp'].iloc[-1])-pd.Timedelta('2s')

    # Get the data only between start_timestamp and end_timestamp
    df = df[start_timestamp:end_timestamp]

    # Make start_timestamp as datetime format
    df_label.loc[:, 'start_timestamp'] = pd.to_datetime(df_label.loc[:,'start_timestamp'])

    # Make the start_timestamp as index
    df_label.set_index('start_timestamp', inplace=True)

    df_label = df_label.resample('20ms').ffill()
    
    # Concatenate the df_label with the df
    df_combined = pd.concat([df, df_label], axis=1)

    # Divide 'x', 'y', 'z' columns by 64
    df_combined.loc[:, ['x', 'y', 'z']] = df_combined.loc[:, ['x', 'y', 'z']] / 64

    # Add a column for user_id
    df_combined['user_id'] = i+1

    # Get timestamp column
    df_combined['timestamp'] = df_combined.index

    # Append to the dataframe
    df_all = pd.concat([df_all, df_combined.loc[:, ['user_id', 'x', 'y', 'z', 'met_value_mean_values', 'timestamp']]])

# Save to CSV file
df_all.to_csv(f'{path}/combined_e4_acc.csv', index=False)